In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\train.parquet')
oos = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\oos.parquet')
oot = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\oot.parquet')

In [3]:
df.columns

Index(['C_GIOITINH', 'TTHONNHAN', 'NHANVIENBIDV', 'BASE_AUM', 'TUOI', 'INCOME',
       'CBAL', 'AFLIMT_AVG', 'LTV', 'N_AVG_DEPOSIT_12M', 'FLAG_SALARY_ACC',
       'FLAG_DEPOSIT', 'UTILIZATION_RATE', 'CNT_CREDIT_CARDS',
       'AMT_CASH_ADVANCE_12M', 'PCT_PAYMENT_TO_BALANCE', 'CNT_MIN_PAY_6M',
       'AVG_DAYS_PAST_DUE', 'DTI_RATIO', 'MOB', 'CNT_OTHER_PRODUCTS',
       'LIMIT_TO_INCOME', 'AMT_VAR_6M', 'CBAL_SHORTTERM_LOAN',
       'CBAL_LONGTERM_LOAN', 'HAS_LONGTERM_LOAN', 'CNT_DPD_30PLUS_6M',
       'OCCUPATION_TYPE', 'DURATION_MAX', 'REMAINING_DURATION_MAX',
       'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX', 'MAX_DPD_12M',
       'AVG_OD_DPD_12M', 'MAX_NHOMNOCIC', 'N_AVG_OVERDUE_CBAL_12M',
       'BAD_NEXT_12M'],
      dtype='str')

In [4]:
df.shape 

(1137807, 38)

In [5]:
# Danh sách các biến cần ép kiểu về Category (Nominal & Binary)
categorical_cols = [
    'OCCUPATION_TYPE', 
    'PURCOD_MAX', 
    'PURCOD_MIN',
    'SOHUUNHA', 
    'NHANVIENBIDV',
    'FLAG_SALARY_ACC', 
    'FLAG_DEPOSIT', 
    'FLAG_CASH_ADVANCE',
    'HAS_SHORTTERM_LOAN', # Nếu có trong df final
    'HAS_LONGTERM_LOAN',  # Nếu có trong df final
    'BAD_CURRENT',        # Nếu giữ lại làm feature (thường là drop vì data leakage)
    'XULYNO'              # Nếu giữ lại
]

# Lọc những cột thực sự tồn tại trong df (đề phòng bạn đã drop bớt)
existing_cat_cols = [col for col in categorical_cols if col in df.columns]

# Chuyển đổi
for col in existing_cat_cols:
    df[col] = df[col].astype('str') # Chuyển về string để EBM hiểu là category
    oos[col] = oos[col].astype('str')
    oot[col] = oot[col].astype('str')

In [6]:
y_train = df['BAD_NEXT_12M']
y_oos = oos['BAD_NEXT_12M']
y_oot = oot['BAD_NEXT_12M']
X = df.drop(['BAD_NEXT_12M', 'HAS_LONGTERM_LOAN', 'AVG_OD_DPD_12M'], axis=1)
X_oos = oos.drop(['BAD_NEXT_12M', 'HAS_LONGTERM_LOAN', 'AVG_OD_DPD_12M'], axis=1)
X_oot = oot.drop(['BAD_NEXT_12M', 'HAS_LONGTERM_LOAN', 'AVG_OD_DPD_12M'], axis=1)

In [7]:
numeric_cols = X.select_dtypes(include=[np.number]).columns
corr_matrix = X[numeric_cols].corr().abs() # Lấy trị tuyệt đối (âm hay dương đều là tương quan mạnh)

# 2. Chọn ngưỡng cắt (Threshold)
# Với EBM, ngưỡng an toàn là 0.7. Ngưỡng 0.8 là bắt buộc phải xử lý.
threshold = 0.7

# 3. Lọc ra các cặp biến vượt ngưỡng
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# Tạo danh sách chi tiết để bạn dễ quyết định giữ ai/bỏ ai
high_corr_pairs = []
for col in upper.columns:
    high_cols = upper.index[upper[col] > threshold].tolist()
    for row in high_cols:
        val = upper.loc[row, col]
        high_corr_pairs.append((row, col, val))

# Sắp xếp theo độ tương quan giảm dần
high_corr_pairs.sort(key=lambda x: x[2], reverse=True)

# 4. In kết quả
print(f"Tìm thấy {len(high_corr_pairs)} cặp biến tương quan cao (> {threshold}):")
print("-" * 60)
print(f"{'Biến 1 (Giữ?)':<30} | {'Biến 2 (Bỏ?)':<30} | {'Corr':<10}")
print("-" * 60)
for pair in high_corr_pairs:
    print(f"{pair[0]:<30} | {pair[1]:<30} | {pair[2]:.4f}")

Tìm thấy 9 cặp biến tương quan cao (> 0.7):
------------------------------------------------------------
Biến 1 (Giữ?)                  | Biến 2 (Bỏ?)                   | Corr      
------------------------------------------------------------
INCOME                         | AFLIMT_AVG                     | 0.8976
UTILIZATION_RATE               | DTI_RATIO                      | 0.8694
BASE_AUM                       | N_AVG_DEPOSIT_12M              | 0.7862
DURATION_MAX                   | REMAINING_DURATION_MAX         | 0.7833
CBAL                           | AFLIMT_AVG                     | 0.7658
DURATION_MAX                   | TIME_TO_OP_MAX                 | 0.7511
MAX_DPD_12M                    | MAX_NHOMNOCIC                  | 0.7483
CBAL                           | N_AVG_OVERDUE_CBAL_12M         | 0.7232
CBAL                           | DTI_RATIO                      | 0.7229


Trong các mô hình dựa trên cây (Tree-based) như EBM, XGBoost, LightGBM, ngưỡng xử lý đa cộng tuyến thường lỏng hơn nhiều so với Logistic Regression. Ngưỡng vàng: Thường là 0.9 hoặc 0.95. Nếu tương quan < 0.9, mô hình vẫn đủ sức phân biệt được tín hiệu riêng biệt của từng biến.

In [9]:
import pandas as pd
import numpy as np

# ============================================================
# 1. DANH SÁCH CÁC CẶP BIẾN CẦN KIỂM TRA
# ============================================================
high_corr_pairs = [
    ('INCOME', 'AFLIMT_AVG'),
    ('UTILIZATION_RATE', 'DTI_RATIO'),
    ('BASE_AUM', 'N_AVG_DEPOSIT_12M'),
    ('DURATION_MAX', 'REMAINING_DURATION_MAX'),
    ('CBAL', 'AFLIMT_AVG'),
    ('DURATION_MAX', 'TIME_TO_OP_MAX'),
    ('MAX_DPD_12M', 'MAX_NHOMNOCIC'),
    ('CBAL', 'N_AVG_OVERDUE_CBAL_12M'),
    ('CBAL', 'DTI_RATIO')
]

# ============================================================
# 2. HÀM TÍNH IV (INFORMATION VALUE) & MISSING RATE
# ============================================================
def calculate_metrics(df, target, feature):
    # 1. Tính Missing Rate
    missing_rate = df[feature].isna().mean()
    
    # 2. Tính Unique values (Độ mịn)
    n_unique = df[feature].nunique()
    
    # 3. Tính IV (Phiên bản đơn giản dùng qcut)
    try:
        # Tạo df tạm
        dff = pd.DataFrame({'feature': df[feature], 'target': target})
        dff = dff.dropna()
        
        # Nếu là biến số -> Binning thành 10 phần
        if pd.api.types.is_numeric_dtype(dff['feature']) and n_unique > 10:
            dff['bin'] = pd.qcut(dff['feature'].rank(method='first'), 10, duplicates='drop')
        else:
            dff['bin'] = dff['feature'].astype(str)
            
        # Group by bin
        grouped = dff.groupby('bin')['target'].agg(['count', 'sum'])
        grouped['non_event'] = grouped['count'] - grouped['sum']
        
        # Tính %
        grouped['pct_event'] = (grouped['sum'] + 0.5) / grouped['sum'].sum() # +0.5 để tránh log(0)
        grouped['pct_non_event'] = (grouped['non_event'] + 0.5) / grouped['non_event'].sum()
        
        # WOE & IV
        grouped['woe'] = np.log(grouped['pct_non_event'] / grouped['pct_event'])
        grouped['iv'] = (grouped['pct_non_event'] - grouped['pct_event']) * grouped['woe']
        
        iv_value = grouped['iv'].sum()
    except Exception as e:
        iv_value = 0.0 # Lỗi thì trả về 0

    return iv_value, missing_rate, n_unique

# ============================================================
# 3. CHẠY VÒNG LẶP SO SÁNH
# ============================================================
print(f"{'VAR 1':<25} | {'IV 1':<6} | {'Miss 1':<6} || {'VAR 2':<25} | {'IV 2':<6} | {'Miss 2':<6} || {'ĐỀ XUẤT GIỮ'}")
print("-" * 115)

# Tập hợp tất cả các biến cần tính (để không tính lại nhiều lần)
all_vars = set([p[0] for p in high_corr_pairs] + [p[1] for p in high_corr_pairs])
metrics_dict = {}

# Tính toán trước cho tất cả biến
print(">>> Đang tính toán chỉ số cho các biến...")
for col in all_vars:
    if col in X.columns:
        metrics_dict[col] = calculate_metrics(X, y_train, col)
    else:
        metrics_dict[col] = (0, 1.0, 0) # Không tìm thấy biến

# So sánh từng cặp
recommendations = []

for v1, v2 in high_corr_pairs:
    iv1, miss1, u1 = metrics_dict.get(v1, (0, 0, 0))
    iv2, miss2, u2 = metrics_dict.get(v2, (0, 0, 0))
    
    # Logic Đề xuất:
    # 1. Nếu Missing chênh lệch quá lớn (> 20%) -> Chọn biến ít Missing hơn
    if abs(miss1 - miss2) > 0.2:
        winner = v1 if miss1 < miss2 else v2
        reason = "Data Quality"
    # 2. Nếu không, chọn biến có IV cao hơn
    elif iv1 > iv2:
        winner = v1
        reason = "Higher IV"
    else:
        winner = v2
        reason = "Higher IV"
        
    # Logic đặc biệt cho trường hợp DPD vs NHOMNOCIC (Ưu tiên độ mịn)
    if "DPD" in v1 and "NHOM" in v2: winner = v1; reason = "Granularity"
    if "DPD" in v2 and "NHOM" in v1: winner = v2; reason = "Granularity"

    print(f"{v1:<25} | {iv1:.4f} | {miss1:.1%}   || {v2:<25} | {iv2:.4f} | {miss2:.1%}   || --> {winner} ({reason})")
    
    # Lưu biến bị loại bỏ để lọc sau này
    loser = v2 if winner == v1 else v1
    recommendations.append(loser)

print("-" * 115)
print(f"\n>>> Gợi ý các biến nên LOẠI BỎ ({len(set(recommendations))} biến):")
print(list(set(recommendations)))

VAR 1                     | IV 1   | Miss 1 || VAR 2                     | IV 2   | Miss 2 || ĐỀ XUẤT GIỮ
-------------------------------------------------------------------------------------------------------------------
>>> Đang tính toán chỉ số cho các biến...
INCOME                    | 0.3391 | 5.0%   || AFLIMT_AVG                | 0.1496 | 0.0%   || --> INCOME (Higher IV)
UTILIZATION_RATE          | 0.1537 | 0.0%   || DTI_RATIO                 | 0.1800 | 0.0%   || --> DTI_RATIO (Higher IV)
BASE_AUM                  | 0.4986 | 0.0%   || N_AVG_DEPOSIT_12M         | 0.9634 | 0.0%   || --> N_AVG_DEPOSIT_12M (Higher IV)
DURATION_MAX              | 0.0003 | 0.0%   || REMAINING_DURATION_MAX    | 0.0001 | 0.0%   || --> DURATION_MAX (Higher IV)
CBAL                      | 0.0136 | 0.0%   || AFLIMT_AVG                | 0.1496 | 0.0%   || --> AFLIMT_AVG (Higher IV)
DURATION_MAX              | 0.0003 | 0.0%   || TIME_TO_OP_MAX            | 0.0002 | 0.0%   || --> DURATION_MAX (Higher IV)
MAX_

In [11]:
X.drop(['UTILIZATION_RATE', 'MAX_NHOMNOCIC', 'REMAINING_DURATION_MAX', 'AFLIMT_AVG', 'BASE_AUM', 'CBAL', 'TIME_TO_OP_MAX'], axis=1, inplace=True)
X_oos.drop(['UTILIZATION_RATE', 'MAX_NHOMNOCIC', 'REMAINING_DURATION_MAX', 'AFLIMT_AVG', 'BASE_AUM', 'CBAL', 'TIME_TO_OP_MAX'], axis=1, inplace=True)
X_oot.drop(['UTILIZATION_RATE', 'MAX_NHOMNOCIC', 'REMAINING_DURATION_MAX', 'AFLIMT_AVG', 'BASE_AUM', 'CBAL', 'TIME_TO_OP_MAX'], axis=1, inplace=True)

In [12]:
X.to_parquet('X_train.parquet', index=False)
X_oos.to_parquet('X_oos.parquet', index=False)
X_oot.to_parquet('X_oot.parquet', index=False)
y_train.to_frame().to_parquet('y_train.parquet', index=False)
y_oos.to_frame().to_parquet('y_oos.parquet', index=False)
y_oot.to_frame().to_parquet('y_oot.parquet', index=False)